In [7]:
%cd ~/google_drive/code/projects/sparse/python
from sparse.core.sparse_dataframe import SparseDataFrame
from sparse.utilities.utils import *

%cd ~/google_drive/code/projects/texture_classifier/python
# %cd /home/ubuntu/texture_classifier/python
from PIL import Image
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist, squareform
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
import cv2
# from plotly import plotly
# from plotly.graph_objs import *

import core.utils
reload(core.utils)
from core.utils import *

import core.image_scanner
reload(core.image_scanner)
from core.image_scanner import ImageScanner

def _random_scan(item, params):
    return [x for x in ImageScanner(Image.open(item), **params).random_scan(params['patches'])]

def _pil_to_opencv(item):
    return cv2.cvtColor(cv2.cvtColor(np.array(item), cv2.COLOR_RGB2BGR), cv2.COLOR_BGR2RGB)

def get_data(info):
    data = info.copy()
    
    temp = data[['source', 'params']].apply(lambda x: (x['source'], x['params']), axis=1)
    data.source = temp.apply(lambda x: _random_scan(x[0], x[1]))
    
    def func(item):
        output = DataFrame()
        output['source'] = item[0]
        output['y'] = item[1]
        output['params'] = None
        output.params = output.params.apply(lambda x: item[2])
        return output

    sdf = SparseDataFrame(data)
    data = sdf.merge_columns(['source', 'common_name', 'params'], func=func, new_column='new')
    data = pd.concat(data.new.tolist(), ignore_index=True)
    
    data['rgb'] = data.source.apply(_pil_to_opencv)
    data['hsv'] = data.rgb.apply(lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2HSV))
    
    temp = data[['rgb', 'params']].apply(lambda x: (x['rgb'], x['params']), axis=1)
    data['r'] = temp.apply(lambda x: get_channel_histogram(x[0], 'r', **x[1])).apply(lambda x: x.tolist())
    data['g'] = temp.apply(lambda x: get_channel_histogram(x[0], 'g', **x[1])).apply(lambda x: x.tolist())
    data['b'] = temp.apply(lambda x: get_channel_histogram(x[0], 'b', **x[1])).apply(lambda x: x.tolist())

    temp = data[['hsv', 'params']].apply(lambda x: (x['hsv'], x['params']), axis=1)
    data['h'] = temp.apply(lambda x: get_channel_histogram(x[0], 'h', **x[1])).apply(lambda x: x.tolist())
    data['s'] = temp.apply(lambda x: get_channel_histogram(x[0], 's', **x[1])).apply(lambda x: x.tolist())
    data['v'] = temp.apply(lambda x: get_channel_histogram(x[0], 'v', **x[1])).apply(lambda x: x.tolist())

    sdf = SparseDataFrame(data)
    data = sdf.flatten(dtype=list)
    
    data = data.drop(['source', 'params', 'rgb', 'hsv'], axis=1)
    return data

/Users/alexbraun/google_drive/code/projects/sparse/python
/Users/alexbraun/google_drive/code/projects/texture_classifier/python


In [2]:
source = '/Users/alexbraun/Documents/data/texture_classifier/data/png'
# source = '/home/ubuntu/texture_classifier/data/png'
spec = ['texture', 'image_id', 'image_class', 'common_name', 'pass_', 'source_extension']

info = get_info(source, spec)
info = info[(info.pass_ == 'diffuse') & (info.image_class == 'a')]
info.reset_index(drop=True, inplace=True)
info = info.head()

params = {
            'min_resolution':   (500, 500),
            'max_resolution':   (500, 500),
            'patches':                 100,
            'patch_resolution': (500, 500),
            'normalize': False,
            'bins': 256
#             'rotation':         'random'
}
info['params'] = None
info.params = info.params.apply(lambda x: params)

data = get_data(info)

In [ ]:
# RGB
for image in images:
    plot_histograms(image)

In [ ]:
# NORMALIZED RGB
for image in images:
    plot_histograms(image, normalize=True)

In [ ]:
# HSV
for image in images:
    plot_histograms(cv2.cvtColor(image, cv2.COLOR_RGB2HSV))

In [ ]:
# NORMALIZED HSV
for image in images:
    plot_histograms(cv2.cvtColor(image, cv2.COLOR_RGB2HSV), normalize=True)

In [ ]:
# RED
for image in images:
    plot_channel_histogram(image, 'r')
    
# 10 distinct reds

In [ ]:
# GREEN
for image in images:
    plot_channel_histogram(image, 'g')
    
# 2 to 8 distinct greens

In [ ]:
# BLUE
for image in images:
    plot_channel_histogram(image, 'b')

# 3 or 5 distinct blues

In [ ]:
for image in hsv:
    plot_channel_histogram(image, 'v', normalize=True)

In [ ]:
X = data.drop('y', axis=1)
y = data.y

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)

clf = RandomForestClassifier()
params = {
#     'n_estimators':        [10]
#     'criterion':           ['gini'],
#     'max_depth':           [None],
#     'min_samples_split':   [2],
#     'min_samples_leaf':    [1],
#     'max_features':        ['auto'],
#     'max_leaf_nodes':      [None],
#     'bootstrap':           [True],
#     'oob_score':           [False],
#     'n_jobs':              [-1]
#     'random_state':        [42]
#     'verbose':             [0],
#     'min_density':         [None],
#     'compute_importances': [None]
}
grid = GridSearchCV(clf, params)
grid.fit(train_x, train_y)
# print(grid.best_score_)
# classification_report()

In [5]:
layout = {
    u'autosize': True,
    u'font': {
        u'color': u'rgb(217, 217, 217)'
    },
    u'margin': {
        u'autoexpand': True,
        u'b': 40,
        u'l': 40,
        u'r': 40,
        u't': 40},
    u'paper_bgcolor': u'rgb(36, 36, 36)',
    u'scene': {
        u'aspectratio': {
            u'x': 1, 
            u'y': 1, 
            u'z': 1
        },
        u'bgcolor': u'rgb(36, 36, 36)',
        u'xaxis': {
            u'gridcolor': u'rgb(146, 178, 222)',
            u'gridwidth': 1.3,
            u'showgrid': True,
            u'showticklabels': True,
            u'type': u'linear',
            u'zeroline': False,
            u'zerolinecolor': u'rgb(255, 255, 255)'
        },
        u'yaxis': {
            u'gridcolor': u'rgb(146, 178, 222)',
            u'gridwidth': 1.3,
            u'showgrid': True,
            u'showticklabels': True,
            u'type': u'linear',
            u'zeroline': False,
            u'zerolinecolor': u'rgb(255, 255, 255)'
        },
        u'zaxis': {
            u'gridcolor': u'rgb(146, 178, 222)',
            u'gridwidth': 1.3,
            u'showgrid': True,
            u'showticklabels': True,
            u'type': u'linear',
            u'zeroline': False,
            u'zerolinecolor': u'rgb(255, 255, 255)'
        }
    },
    u'showlegend': False,
    u'titlefont': {
        u'color': u'rgb(167, 202, 207)'
    }
 }

style_data = {
    u'marker': {
        u'color': u'rgba(146, 178, 222, 0.76)',
        u'symbol': u'diamond-open'
    },
    u'mode': u'markers',
    u'name': u'y',
    u'opacity': 0.45,
    u'projection': {
        u'x': {u'show': False},
        u'y': {u'show': False},
        u'z': {u'show': False}
    },
    u'surfaceaxis': -1
}

In [ ]:
np.dstack((r,g,b)).shape

In [8]:
layout['height'] = 700
layout['width'] = 700
layout['scene']['xaxis']['title'] = 'hue'
layout['scene']['yaxis']['title'] = 'saturation'
layout['scene']['zaxis']['title'] = 'value'
# x = Scatter3d(x=r, y=g, z=b, **style_data)
x = Scatter3d(x=h, y=s, z=v, **style_data)
x = Data([x])
fig = Figure(data=x, layout=layout)
plotly.iplot(fig)

NameError: name 'Scatter3d' is not defined

In [ ]:
def get_color_data(source, patches=50):
    img = Image.open(source)
    
    img = ImageScanner(img, **params)
    datum = []
    for patch in img.random_scan(patches):
        patch = np.array(patch)
        patch = patch.reshape(patch.shape[0] * patch.shape[1], 3)
        datum.append(patch)
    return np.concatenate(datum, axis=0)

data = info['source'].apply(lambda x: get_color_data(x, 10))
X = np.concatenate(data, axis=0)
X = DataFrame(X, columns=['r', 'g', 'b'])

In [ ]:
info = get_info(source, spec)
info = info[(info.pass_ == 'diffuse') & (info.image_class == 'a')]
info.reset_index(drop=True, inplace=True)
# info = info.head(2)

data = []
for i, row in info.iterrows():
    img = Image.open(row['source'])
    img = ImageScanner(img, **params)
    datum = []
    for patch in img.random_scan(50):
        patch = np.array(patch)
        patch = patch.reshape(patch.shape[0] * patch.shape[1], 3)
        datum.append(patch)
    datum = np.concatenate(datum, axis=0)
    data.append(datum)
data = np.concatenate(data, axis=0)
data.shape

In [ ]:
X = data
clf = KMeans(n_clusters=10, n_jobs=-1)
%time clf.fit(X)

In [ ]:
clf.cluster_centers_

In [ ]:
y = clf.cluster_centers_

fig, ax = plt.subplots(figsize=(50,50))

pw_dist = squareform(pdist(X))
h_clusters = linkage(pw_dist)

ax = fig.add_subplot(221)
dendrogram(h_clusters,
    ax=ax,
    color_threshold=1.89,
    color_list=colors,
    labels=data.headline.values,
    leaf_font_size=14,
#     leaf_rotation=90,
    orientation='left')

pw_dist = squareform(pdist(X.toarray(), metric='cosine'))
h_clusters = linkage(pw_dist)
# plt.figure(figsize=(50,50))
ax = fig.add_subplot(222)
dendrogram(h_clusters,
    ax=ax,
    color_threshold=1.3,
    color_list=colors,
    labels=data.headline.values,
    leaf_font_size=14,
#     leaf_rotation=90,
    orientation='left')
plt.tight_layout()
plt.show();

In [ ]:
# info['scan_params'] = None
# info.scan_params = info.scan_params.apply(lambda x: params)

# def func(item):
#     item['scan_params']['image'] = item['source']
#     return item[1]
# info[['source', 'scan_params']].apply(func, axis=1).head(1).values

In [ ]:
import os
import inspect
# _script = inspect.currentframe()
# _script = inspect.getfile(_script)
# _script = os.path.abspath(_script)
# _script = os.path.dirname(_script)
# _script = os.path.join(_script, 'to_format')

def get_target(fullpath, format_):
    base, filename = os.path.split(fullpath)
    base = os.path.dirname(base)
    filename = os.path.splitext(filename)[0] + '.' + format_
    return os.path.join(base, format_, filename)

def create_msgpacks(root):
    args = [os.path.join(root, x) for x in os.listdir(root)]
    args = [(x, get_target(x, 'msgpack')) for x in args]
    target = os.path.split(args[0][1])[0]
    if not os.path.exists(target):
        os.mkdir(target)
    execute_python_subshells(_script, args)